In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import os

import numpy as np
import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.module
import strawb.tools

import h5py 

import pandas

# Load ONC DB and mask files of interest

In [ ]:
# in case execute db.load_onc_db_update to load the entire db
if os.path.exists(strawb.Config.pandas_file_sync_db):
    db = strawb.SyncDBHandler()  # loads the db
else:
    db = strawb.SyncDBHandler(load_db=False)  # loads the db
    db.load_onc_db_update(output=True, save_db=True)

# mask by device
mask = db.dataframe['deviceCode'] == 'TUMMUONTRACKER001'
mask &= db.dataframe.dataProductCode == 'MTSD'  # see SyncDBHandler.sensor_mapping

In [ ]:
# show some masked entries
db.dataframe[mask].iloc[-5:]

# Download a specific file, needs some minutes if file isn't synced

In [ ]:
file_list = ['TUMMUONTRACKER001_20220731T001146.146Z-SDAQ-MUON.hdf5']
db_i = db.get_files_from_names(file_list)
db_i

# Load the file

In [ ]:
muon = strawb.MuonTracker(file_list[0])

## show file version - 4 has tot data, which we need

In [ ]:
muon.file_handler.file_version

## Show data structure

In [ ]:
# all have the same shape -> entry i represents one event
print(f'tot_time   : {muon.file_handler.tot_time.shape}')
print(f'tot_time_ns: {muon.file_handler.tot_time_ns.shape}')
print(f'tot_channel: {muon.file_handler.tot_channel.shape}')
print(f'tot_tot    : {muon.file_handler.tot_tot.shape}')

In [ ]:
# show the first n events
n = 5

# Absolute Timestamp in seconds [s] with resolution [ns]
print(f'tot_time   : {muon.file_handler.tot_time[:n]}')

# Absolute Timestamp in seconds [s] with resolution [ns], converted to dateformat
print(f'tot_time   : {muon.file_handler.tot_time.asdatetime()[:n]}')

# TRB Timestamp in [ns] - not absolute therefore most precise
print(f'tot_time_ns: {muon.file_handler.tot_time_ns[:n]}')

# TRB Channel of the event
print(f'tot_channel: {muon.file_handler.tot_channel[:n]}')

# TRB time over threshold in [ns]
print(f'tot_tot    : {muon.file_handler.tot_tot[:n]}')

# Load and Cut Data

In [ ]:
tot = muon.file_handler.tot_tot[:]

max_tot = 1 * 1e3 # cut at 1us - [ns]

mask = (tot<max_tot) & (tot > 0)

print(f'exclude {np.sum(~mask)} events')

time_masked = muon.file_handler.tot_time[mask]
time_ns_masked = muon.file_handler.tot_time_ns[mask]
channel_masked = muon.file_handler.tot_channel[mask]
tot_masked = muon.file_handler.tot_tot[mask]

# free RAM - parameter not needed
del tot, mask

# Plot

In [ ]:
# timeline of events

plt.figure()

n = 1000
sc = plt.scatter(strawb.tools.asdatetime(time_masked)[:n],
            tot_masked[:n],
            c=channel_masked[:n])

plt.colorbar(sc, label='Channel')

ax = plt.gca()
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator())
)
ax.set_axisbelow(True)  # show grid behind scatter

plt.ylabel('ToT [ns]')
plt.xlabel('Date [ns]')
plt.grid()

## Show short tot's

In [ ]:
# gen. hist
counts, edges = np.histogram(tot_masked, bins=1000)

# plot hist
plt.figure()
plt.stairs(counts, edges=edges)
plt.xlabel('ToT [ns]')
plt.ylabel('Counts [ns]')
plt.yscale('log')